In [11]:
import pandas as pd
from datetime import datetime

In [46]:
imdbRaw = pd.read_csv("/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/Fall 2024/Movie_Creation_Tool-1/ignoredFiles/TMDB_all_movies.csv",usecols=["title", "imdb_id", "release_date"])

In [45]:
imdbRaw

,title,release_date,imdb_id
0,Ariel,1988-10-21,tt0094675
1,Shadows in Paradise,1986-10-17,tt0092149
2,Four Rooms,1995-12-09,tt0113101
3,Judgment Night,1993-10-15,tt0107286
4,Life in Loops (A Megacities RMX),2006-01-01,tt0825671
...,...,...,...
1000021,Centenario,NaT,NaN
1000022,Bonafide - Messin' In Wales,NaT,NaN
1000023,Женщина с цветами и шампанским,NaT,NaN
1000024,NaN,NaT,NaN


In [48]:
# Step 1: Convert 'release_date' to datetime and extract the year
imdbRaw['release_date'] = pd.to_datetime(imdbRaw['release_date'], format='%Y-%m-%d', errors='coerce')

# Step 2: Filter rows where both 'imdb_id' and 'release_date' are valid (non-null)
valid_rows = imdbRaw[pd.notna(imdbRaw['imdb_id']) & pd.notna(imdbRaw['release_date'])].copy()

# Step 3: Extract the year and assign it to a new column 'release_year'
valid_rows['release_year'] = valid_rows['release_date'].dt.year

# Now valid_rows is your "imdbProcessed" dataframe with valid rows and a 'release_year' column
imdbProcessed = valid_rows

# Optional: If you want to reset the index
imdbProcessed.reset_index(drop=True, inplace=True)

In [49]:
imdbProcessed

,title,release_date,imdb_id,release_year
0,Ariel,1988-10-21,tt0094675,1988
1,Shadows in Paradise,1986-10-17,tt0092149,1986
2,Four Rooms,1995-12-09,tt0113101,1995
3,Judgment Night,1993-10-15,tt0107286,1993
4,Life in Loops (A Megacities RMX),2006-01-01,tt0825671,2006
...,...,...,...,...
555877,"Martin Mull Live from North Ridgeville, Ohio",1987-08-15,tt0484861,1987
555878,Freelance,2024-10-12,tt27634066,2024
555879,William S. Burroughs in the Dreamachine,2015-04-14,tt4595996,2015
555880,Nick Broms: What's the rush?,2023-04-25,tt16578844,2023


In [50]:
imdbProcessed.to_csv("/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/Fall 2024/Movie_Creation_Tool-1/ians-playground/imdbProcessed.csv")